### 01. PUBG 승률예측 모델 기본 모델 제작

### 실습 내용 : DecisionClassifierTree를 이용하여 winPlacePerc값을 예측하는 모델 구축

#### 

In [1]:
!pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [78]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_absolute_error

In [25]:
train = pd.read_csv("train_V2.csv")
test = pd.read_csv("test_V2.csv")
sub= pd.read_csv("sample_submission_V2.csv")

In [26]:
train.shape, test.shape

((4446966, 29), (1934174, 28))

In [27]:
train.info(), test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4446966 entries, 0 to 4446965
Data columns (total 29 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Id               object 
 1   groupId          object 
 2   matchId          object 
 3   assists          int64  
 4   boosts           int64  
 5   damageDealt      float64
 6   DBNOs            int64  
 7   headshotKills    int64  
 8   heals            int64  
 9   killPlace        int64  
 10  killPoints       int64  
 11  kills            int64  
 12  killStreaks      int64  
 13  longestKill      float64
 14  matchDuration    int64  
 15  matchType        object 
 16  maxPlace         int64  
 17  numGroups        int64  
 18  rankPoints       int64  
 19  revives          int64  
 20  rideDistance     float64
 21  roadKills        int64  
 22  swimDistance     float64
 23  teamKills        int64  
 24  vehicleDestroys  int64  
 25  walkDistance     float64
 26  weaponsAcquired  int64  
 27  winPoints   

(None, None)

In [28]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1934174 entries, 0 to 1934173
Data columns (total 28 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Id               object 
 1   groupId          object 
 2   matchId          object 
 3   assists          int64  
 4   boosts           int64  
 5   damageDealt      float64
 6   DBNOs            int64  
 7   headshotKills    int64  
 8   heals            int64  
 9   killPlace        int64  
 10  killPoints       int64  
 11  kills            int64  
 12  killStreaks      int64  
 13  longestKill      float64
 14  matchDuration    int64  
 15  matchType        object 
 16  maxPlace         int64  
 17  numGroups        int64  
 18  rankPoints       int64  
 19  revives          int64  
 20  rideDistance     float64
 21  roadKills        int64  
 22  swimDistance     float64
 23  teamKills        int64  
 24  vehicleDestroys  int64  
 25  walkDistance     float64
 26  weaponsAcquired  int64  
 27  winPoints   

In [29]:
train.head(),test.head()

(               Id         groupId         matchId  assists  boosts  \
 0  7f96b2f878858a  4d4b580de459be  a10357fd1a4a91        0       0   
 1  eef90569b9d03c  684d5656442f9e  aeb375fc57110c        0       0   
 2  1eaf90ac73de72  6a4a42c3245a74  110163d8bb94ae        1       0   
 3  4616d365dd2853  a930a9c79cd721  f1f1f4ef412d7e        0       0   
 4  315c96c26c9aac  de04010b3458dd  6dc8ff871e21e6        0       0   
 
    damageDealt  DBNOs  headshotKills  heals  killPlace  ...  revives  \
 0         0.00      0              0      0         60  ...        0   
 1        91.47      0              0      0         57  ...        0   
 2        68.00      0              0      0         47  ...        0   
 3        32.90      0              0      0         75  ...        0   
 4       100.00      0              0      0         45  ...        0   
 
    rideDistance  roadKills  swimDistance  teamKills vehicleDestroys  \
 0        0.0000          0          0.00          0        

In [30]:
train.columns

Index(['Id', 'groupId', 'matchId', 'assists', 'boosts', 'damageDealt', 'DBNOs',
       'headshotKills', 'heals', 'killPlace', 'killPoints', 'kills',
       'killStreaks', 'longestKill', 'matchDuration', 'matchType', 'maxPlace',
       'numGroups', 'rankPoints', 'revives', 'rideDistance', 'roadKills',
       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance',
       'weaponsAcquired', 'winPoints', 'winPlacePerc'],
      dtype='object')

In [41]:
train.info(null_counts=True)

C:\Users\이길원\AppData\Local\Temp\ipykernel_8080\3266938051.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  train.info(null_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4446966 entries, 0 to 4446965
Data columns (total 29 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   Id               4446966 non-null  object 
 1   groupId          4446966 non-null  object 
 2   matchId          4446966 non-null  object 
 3   assists          4446966 non-null  int64  
 4   boosts           4446966 non-null  int64  
 5   damageDealt      4446966 non-null  float64
 6   DBNOs            4446966 non-null  int64  
 7   headshotKills    4446966 non-null  int64  
 8   heals            4446966 non-null  int64  
 9   killPlace        4446966 non-null  int64  
 10  killPoints       4446966 non-null  int64  
 11  kills            4446966 non-null  int64  
 12  killStreaks      4446966 non-null  int64  
 13  longestKill      4446966 non-null  float64
 14  matchDuration    4446966 non-null  int64  
 15  matchType        4446966 non-null  object 
 16  maxPlace         4

In [43]:
train.dropna(axis=0, inplace=True)
train.info(null_counts=True)

C:\Users\이길원\AppData\Local\Temp\ipykernel_8080\403389399.py:2: FutureWarning: null_counts is deprecated. Use show_counts instead
  train.info(null_counts=True)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4446965 entries, 0 to 4446965
Data columns (total 29 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   Id               4446965 non-null  object 
 1   groupId          4446965 non-null  object 
 2   matchId          4446965 non-null  object 
 3   assists          4446965 non-null  int64  
 4   boosts           4446965 non-null  int64  
 5   damageDealt      4446965 non-null  float64
 6   DBNOs            4446965 non-null  int64  
 7   headshotKills    4446965 non-null  int64  
 8   heals            4446965 non-null  int64  
 9   killPlace        4446965 non-null  int64  
 10  killPoints       4446965 non-null  int64  
 11  kills            4446965 non-null  int64  
 12  killStreaks      4446965 non-null  int64  
 13  longestKill      4446965 non-null  float64
 14  matchDuration    4446965 non-null  int64  
 15  matchType        4446965 non-null  object 
 16  maxPlace         4

In [65]:
X_tr_all = train.loc[ : , ['assists','kills','killPoints','killStreaks','DBNOs']]
label_name = 'winPlacePerc'
train = train.astype({'winPlacePerc':'int'})
y_tr_all = train[label_name]


In [66]:
X_tr_all.shape
y_tr_all.shape

(4446965,)

In [67]:
y_tr_all.info()

<class 'pandas.core.series.Series'>
Int64Index: 4446965 entries, 0 to 4446965
Series name: winPlacePerc
Non-Null Count    Dtype
--------------    -----
4446965 non-null  int32
dtypes: int32(1)
memory usage: 50.9 MB


In [70]:
y_tr_all.value_counts()

0    4319392
1     127573
Name: winPlacePerc, dtype: int64

In [71]:
X_train,X_test,y_train,y_test = train_test_split(X_tr_all, y_tr_all, random_state=33, test_size = 0.2)

In [72]:
y_train.info()

<class 'pandas.core.series.Series'>
Int64Index: 3557572 entries, 2613096 to 1989652
Series name: winPlacePerc
Non-Null Count    Dtype
--------------    -----
3557572 non-null  int32
dtypes: int32(1)
memory usage: 40.7 MB


In [86]:
model = DecisionTreeClassifier()
model = model.fit(X_train,y_train)
pred = model.predict_proba(X_test)[:,1]
pred

array([0.00737101, 0.02184044, 0.        , ..., 0.01015954, 0.00195312,
       0.01015954])

In [85]:
pred[:,1],y_test.head()

(array([0.00737101, 0.02184044, 0.        , ..., 0.01015954, 0.00195312,
        0.01015954]),
 2579961    0
 2025855    0
 2081019    0
 625211     0
 2944871    0
 Name: winPlacePerc, dtype: int32)

In [89]:
type( y_test.values )

numpy.ndarray

In [90]:
print("MAE : ",mean_absolute_error(y_test.values,pred))

MAE :  0.047060624884740804
